# Modulación LoRa

## Concepto Principal

La modulación LoRa es una de las maneras de transmitir mensajes digitales a través de técnicas de modulación de **espectro expandido**. $^1$ 

Específicamente, este tipo de modulación es una técnica de espectro expandido con codificación a base de chirps (CSS). **Un chirp** es aquella señal en la cual va aumentando o disminuyendo su frecuencia a lo largo del tiempo. En este caso, se trabaja con un chirp con barrido de frecuencia lineal, por lo que la frecuencia instantánea será de la forma:

 $y = b + m\cdot t$

Para luego transmitir la señal como una exponencial compleja, modulada en fase en función de la acumulación de frecuencia instantánea:

$y(t) = c \cdot e^{2\pi \int_{0}^{t}f(\tau)d\tau}$

### Distinguiendo símbolos

La manera que tiene LoRa de diferenciar entre símbolos radica simplemente en definir a la ordenada al origen como una función del símbolo a enviar, haciendo que este esté implicito en la forma de onda resultante. La completa definición se verá en la sección del desarrollo matemático.

$^1$ La modulación de espectro expandido es aquel tipo de modulación en el cual la señal a transmitir está dispersa en el campo de la frecuencia en mayor medida de la necesaria para transmitir información. De esta forma, se permite construir técnicas de modulación con mayor resistencia al ruido e interferencias. Además, la potencia también está mejor distribuida en estas frecuencias, asemejándose a la densidad espectral de potencia del ruido, por lo que tiene también importancia en la seguridad. Que la PSD de la señal sea bien distribuida, implica que la autocorrelación solo estará presente cuando no hay retardos en el tiempo, por lo que muestrear la señal sincronizada es muy importante.




## Parámetros Externos para el MoDem

En esta sección se explorarán los parámetros que definen como funcionará nuestro MoDem LoRa.

- Spreading Factor ($SF$): El factor de expansión de una modulación LoRa por sí mismo solo define explícitamente la cantidad de bits contenidos en un símbolo. Sin embargo, su parámetro interno ($2^{SF}$) nos permitirá entender más conceptos asociados a la modulación (se aborda más adelante). Los valores típicos son: [7, 8, 9, 10, 11, 12]
- Bandwidth ($Bw$): El ancho de banda define los valores en frecuencia que limitarán al barrido del chirp y a su vez es un factor determinante en la duración de un símbolo. Los valores típicos son: [125kHz, 250kHz, 500kHz]
- Samples per Chip ($spc$): La cantidad de muestras por chip o *"trozo"* $^{2}$. Representa la cantidad de muestras que cada chip de tiempo tendrá.

$^2$ Para entneder lo que es un chip en la modulación LoRa, se puede explorar de donde se hereda su témrino, o sea de la modulación de espectro expandido de forma directa: En un sistema DSS, cada bit de información que se transmita se multiplicará por un codigo de expansión o secuencia de chips específica, que es donde sucede la expansión característica del espectro. De esta forma, para una frecuencia de llegada de bits no codificados, se tendrá una frecuencia mayor de bits codificados por la secuencia de expansión. A estos bits de mayor tamaño se los conoce como chips.

En el marco de la modulación LoRa, por cada símbolo transmitido se tendrá que computar $2^{SF}$ chips, ya que cada uno ocupará una frecuencia específica. Estos se distribuyen en el tiempo y es entonces como sucede la expansión del espectro.

Si bien uno podría objetar que la cantidad de muestras por chips cambia la cantidad de muestras computadas, esto simplemente repercute en la descripción gráfica de la señal, puesto que este parámetro **no permite que se puedan modular una mayor cantidad de símbolos**, ya que las ordenadas al origen y por lo tanto las discontinuidades del conjunto $\mathcal W$ permanecen iguales y cada una mantiene su valor original sin importar como evoluciona este parámetro.


## Parámetros Internos para el MoDem

En esta sección se abordarán los distintos parámetros de los que se hace uso en repetidas ocasiones, que se originan directamente de los parámetros ya vistos.

- $2^{SF}$: Esta potencia nos indica muchas cuestiones acerca de nuestra modulación a estudiar:
  - Número de símbolos posibles para transmitir: Una modulación LoRa podrá transmitir desde el símbolo 0 hasta el símbolo $2^{SF}-1$. 
  - Número de chips por símbolo: Son el mínimo número de chips temporales necesarios en un símbolo para poder asignarle un valor temporal a cada incremento de frecuencia. Es decir, que como mínimo en un código se tendrán $2^{SF}$ muestras.
  - Número de formas de onda posibles: Para cada símbolo habrá una forma de onda distinta a las otras. Justamente una demodulación propuesta (ineficiente por sus largos tiempos de cómputo) es hacer la correlación de la señal recibida con las $2^{SF}$ señales posibles, y de esa manera asumir que el símbolo recibido sea el asociado a la que logre mayor correlación.
  

- Duración de Símbolo: La duración de símbolo nos permite saber cuanto se tarda para que se complete la transmisión de un símbolo. Se define como el producto de la duración de un solo chip temporal por la cantidad de chips presentes en un símbolo, es decir:
  
  - $T_s = 2^{SF} \cdot \frac{1}{Bw} = \frac{2^{SF}}{Bw}$

- Samples per Symbol: La cantidad total de muestras por símbolo estará dada por el producto de la cantidad de chips por símbolo por la cantidad de muestras por cada chip, a saber:
  
  - $SPS = SPC \cdot 2^{SF}$

- Frequency Slope: Corresponde a la pendiente que tendrá la función frecuencia instantánea a lo largo del tiempo. Por lo tanto, se define como la tasa de cmabio que esta función experimentará, quedando manifestada como lo indica la definición de *pendiente*:
  - $m = \frac{\Delta y}{\Delta t} = \frac{(Bw - 0)}{(T_{s} - 0)} = \frac{(Bw)}{(T_{s})} = \frac{(Bw)^2}{(2^{SF})}$

## Desarrollo Matemático

Para abordar el desarrollo matemático de la modulación LoRa, se partirá desde el análisis de la frecuencia a lo largo del tiempo, lo cual sabemos .

Como vimos anteriormente, un chirp lineal en el tiempo será del tipo:
$f(t) = b + m\cdot t$